In [2]:
import numpy as np
import shutil
import os
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import regions
from astroquery.vizier import Vizier
from astropy.visualization import quantity_support
from astropy import log
from astropy.table import Table
import warnings

from astropy.wcs import WCS
from astropy.io import fits

import matplotlib.pyplot as plt

from align_to_catalogs import realign_to_vvv, realign_to_catalog, merge_a_plus_b, retrieve_vvv

# do this before importing webb==
os.environ["CRDS_PATH"] = "/orange/adamginsburg/jwst/brick/crds/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

from jwst.pipeline import calwebb_image3
from jwst.pipeline import Detector1Pipeline, Image2Pipeline

# Individual steps that make up calwebb_image3
from jwst.tweakreg import TweakRegStep
from jwst.skymatch import SkyMatchStep
from jwst.tweakreg.utils import adjust_wcs
from jwst.outlier_detection import OutlierDetectionStep
from jwst.resample import ResampleStep
from jwst.source_catalog import SourceCatalogStep
from jwst import datamodels
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.tweakreg.utils import adjust_wcs
from destreak import destreak
from jwst.datamodels import ImageModel

In [1]:
fn = '/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_i2d.fits'
fn_realigned = '/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_realigned-to-vvv.fits'

In [15]:
refcat = '/orange/adamginsburg/jwst/cloudc/catalogs/crowdsource_based_nircam-f405n_reference_astrometric_catalog.fits'
vvvcat = '/orange/adamginsburg/jwst/cloudc/catalogs/jw02221-o002_t001_nircam_clear-F405N-merged_vvvcat.fits'

In [16]:
ls /orange/adamginsburg/jwst/cloudc/F187N/pipeline/*clear*nrca*

/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_cat.ecsv
/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_i2d.fits
/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_i2d_satstar_catalog.fits
/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_i2d_unsatstar.fits
/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_nodestreak_realigned-to-refcat.fits
/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_nodestreak_realigned-to-vvv.fits
/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_nodestreak_realigned-to-vvv_satstar_catalog.fits
/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_nodestreak_realigned-to-vvv_unsatstar.fits
/orange/adamginsburg/jwst/cloudc/F187N/pipeline/jw02221-o

In [17]:
hdu = fits.open(fn)

In [18]:
hdu_realigned = fits.open(fn_realigned)

In [19]:
basepath = '/orange/adamginsburg/jwst/cloudc/'

In [23]:
filtername = 'F187N'
fov_regname = {'brick': 'regions_/nircam_brick_fov.reg',
               'cloudc': 'regions_/nircam_cloudc_fov.reg',
              }
regionname = 'cloudc'
field = '002'
proposal_id = '2221'
module = 'nrca'
destreak_suffix = ''
raoffset = 0*u.arcsec
decoffset = 0*u.arcsec
wavelength = 187

In [29]:
realigned_vvv_filename = f'{basepath}/{filtername.upper()}/pipeline/jw0{proposal_id}-o{field}_t001_nircam_clear-{filtername.lower()}-{module}{destreak_suffix}_realigned-to-vvv.fits'
shutil.copy(f'{basepath}/{filtername.upper()}/pipeline/jw0{proposal_id}-o{field}_t001_nircam_clear-{filtername.lower()}-{module}_i2d.fits',
            realigned_vvv_filename)
realigned = realign_to_vvv(filtername=filtername.lower(),
                           fov_regname=fov_regname[regionname],
                           basepath=basepath, module=module,
                           fieldnumber=field, proposal_id=proposal_id,
                           imfile=realigned_vvv_filename,
                           ksmag_limit=15 if filtername.lower() == 'f410m' else 11,
                           mag_limit=18 if filtername.lower() == 'f115w' else 15,
                           max_offset=(0.4 if wavelength > 250 else 0.3)*u.arcsec,
                           raoffset=raoffset,
                           decoffset=-0.1*u.arcsec)

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f187n nrca 002 catalog /orange/adamginsburg/jwst/cloudc//F187N/pipeline/jw02221-o002_t001_nircam_clear-f187n-nrca_cat.ecsv, found 1877 of 26210 sources meeting criteria mag<15 [align_to_catalogs]
2024-01-12T16:26:38.333883: At realignment iteration 0, offset is 0.01683004118149256 arcsec, 0.08969860832337417 arcsec.  Found 1215 matches.
2024-01-12T16:26:38.786245: At realignment iteration 1, offset is 0.00023751614435241208 arcsec, 0.005337756355316969 arcsec.  Found 1258 matches.
2024-01-12T16:26:39.045864: At realignment iteration 2, offset is 5.606441391137196e-05 arcsec, 0.0001256340887323404 arcsec.  Found 1260 matches.
2024-01-12T16:26:39.152075: CRVAL before 266.5905159646875 -28.564360356981712
2024-01-12T16:26:39.176967: CRVAL after 266.59051120813 -28.564359013092
2024-01-12T16:26:39.961667: After realignment, offset is 9.106315701501444e-09 arcsec, 1.7010393094096798e-09 arcsec with 1260

In [3]:
im = ImageModel(fn)

In [4]:
im

<ImageModel(5034, 10598) from jw02221-o002_t001_nircam_clear-f187n-nrca_i2d.fits>